# Test of a way to build custom notebooks from a base and a topic (tag)


First test assuming that the topics are exclusively in independent files, thus a direct indexing of the cells can be appropriate

In [7]:
import json
base_path = "RK-base.ipynb"
with open(base_path, 'r+') as f:
    base_nb = json.load(f)
    
print(base_nb.keys())
#print(nb['cells'])

biology_path = "RK-biology.ipynb"
with open(biology_path, 'r+') as f:
    biology_nb = json.load(f)

for i in range(len(biology_nb['cells'])):
    base_nb['cells'].append(biology_nb['cells'][i])

new_nb_path = "RK-biology_notebook.ipynb"
with open(new_nb_path, 'w') as f:
    json.dump(base_nb, f)


dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])


Second test assuming that all the topics in a single file, different than base, thus a tag validation is required for appending the correct cells

In [ ]:
import json
base_path = "RK-base.ipynb"
with open(base_path, 'r+') as f:
    base_nb = json.load(f)
    
print(base_nb.keys())
#print(nb['cells'])

topics_path = "RK-topics.ipynb"
with open(biology_path, 'r+') as f:
    biology_nb = json.load(f)

for i in range(len(biology_nb['cells'])):
    base_nb['cells'].append(biology_nb['cells'][i])

new_nb_path = "RK-biology_notebook.ipynb"
with open(new_nb_path, 'w') as f:
    json.dump(base_nb, f)


Third test assuming a master notebook with the base and topics, thus a tag validation is required for dropping the cells that are not relevant for the topics

In [ ]:
import json
base_path = "RK-base.ipynb"
with open(base_path, 'r+') as f:
    base_nb = json.load(f)
    
print(base_nb.keys())
#print(nb['cells'])

biology_path = "RK-biology.ipynb"
with open(biology_path, 'r+') as f:
    biology_nb = json.load(f)

for i in range(len(biology_nb['cells'])):
    base_nb['cells'].append(biology_nb['cells'][i])

new_nb_path = "RK-biology_notebook.ipynb"
with open(new_nb_path, 'w') as f:
    json.dump(base_nb, f)


In [8]:
# create topics file

import json

#print(nb['cells'])

biology_path = "RK-biology.ipynb"
with open(biology_path, 'r+') as f:
    biology_nb = json.load(f)
    
chemistry_path = "RK-chemistry.ipynb"
with open(chemistry_path, 'r+') as f:
    chemistry_nb = json.load(f)
    
physics_path = "RK-physics.ipynb"
with open(physics_path, 'r+') as f:
    physics_nb = json.load(f)

for i in range(len(chemistry_nb['cells'])):
    biology_nb['cells'].append(chemistry_nb['cells'][i])
    
for i in range(len(physics_nb['cells'])):
    biology_nb['cells'].append(physics_nb['cells'][i])

new_nb_path = "RK-topics_notebook.ipynb"
with open(new_nb_path, 'w') as f:
    json.dump(biology_nb, f)


In [9]:
# create master file

import json

#print(nb['cells'])
base_path = "RK-base.ipynb"
with open(base_path, 'r+') as f:
    base_nb = json.load(f)

biology_path = "RK-biology.ipynb"
with open(biology_path, 'r+') as f:
    biology_nb = json.load(f)
    
chemistry_path = "RK-chemistry.ipynb"
with open(chemistry_path, 'r+') as f:
    chemistry_nb = json.load(f)
    
physics_path = "RK-physics.ipynb"
with open(physics_path, 'r+') as f:
    physics_nb = json.load(f)

for i in range(len(biology_nb['cells'])):
    base_nb['cells'].append(biology_nb['cells'][i])
    
for i in range(len(chemistry_nb['cells'])):
    base_nb['cells'].append(chemistry_nb['cells'][i])
    
for i in range(len(physics_nb['cells'])):
    base_nb['cells'].append(physics_nb['cells'][i])

new_nb_path = "RK-master_notebook.ipynb"
with open(new_nb_path, 'w') as f:
    json.dump(base_nb, f)

In [8]:
import os
import json
from string import Template


def add_module_badges(fpath):
    """ Looks through all markdown cells in a given notebook for
    those tagged with a module-tag. If found, it adds the module
    badges below the header and appends the {doc} directive to
    that notebook to the modules_dict.

    """

    # Fix fpath for badges, remove _tmp
    new_fpath = fpath.split('_tmp')[-1].split('.ipynb')[0]

    with open(fpath, 'r+') as f:
        nb = json.load(f)

    file_changed = False
    for cell in nb['cells']:

        ind_header = None
        if cell['cell_type'] == 'markdown':
            for i, line in enumerate(cell['source']):

                # find the first header in the cell, under which we will add badges
                _header = line.split("#")
                if ind_header is None and len(_header) > 1:
                    ind_header = i
                    header = _header[-1]

                # remove any previous module badges
                if '<!-- module-' in line:
                    del cell['source'][i]
                    file_changed = True
                    continue

            _module_tags = []
            try:
                tags = cell['metadata']['tags']
                for tag in tags:
                    if "module-" in tag and tag in modules_dict:
                        _module_tags.append(tag)

                # single line cells don't have \n at the end
                if len(_module_tags) > 0:
                    if '\n' not in cell['source'][ind_header]:
                        cell['source'][ind_header] += '\n'

                    # badges and {doc} links to notebook
                    tags_string = ''
                    for tag in _module_tags:
                        _ref = "{doc}" + f"`{header.strip()} <{new_fpath}>`"
                        modules_dict[tag] += [_ref]
                        modules_dict[tag].sort()
                        tags_string += modules_dict[tag][1] + ' '
                    tags_string += '\n'

                    # add badges to the cell content
                    cell['source'].insert(ind_header + 1, tags_string)
                    file_changed = True

                    print(f"Added {len(_module_tags)} badges to {fpath}")

            except KeyError:
                continue

            if file_changed:
                with open(fpath, 'w') as f:
                    json.dump(nb, f)


def write_module_content():
    """ Writes the values in modules_dict dictionary into the
    second cell of modules.ipynb. Creates a ## header based on
    module name and lists links to notebooks containing cells with
    a corresponding module-tag.

    """

    content = []
    for year in modules_year.keys():
        content += [f'## {year}\n']

        for tag in modules_dict.keys():
            if tag in modules_year[year] and len(modules_dict[tag]) > 2:

                # create target header and badge (first two entries of modules_dict)
                content += [f'({tag})=\n', f'### {modules_dict[tag][0]}\n', '\n', f'{modules_dict[tag][1]}\n', '\n']

                for line in modules_dict[tag][2:]:
                    content += ['- ' + line + '\n']
                content += ['\n']

    with open(modules_path, 'r+') as f:
        nb = json.load(f)

    # write content in the second cell
    nb['cells'][1]['source'] = content
    with open(modules_path, 'w') as f:
        json.dump(nb, f)
